# 웹 검색, 검색 엔진, 그리고 파일 검색 API 여정

이 인터랙티브 튜토리얼에 오신 것을 환영합니다! 🚀

## 학습 목표

이 노트북에서는 최신 AI 기반 검색이 어떻게 작동하는지 살펴봅니다. 튜토리얼을 마치면 다음을 이해하게 됩니다:

1. **AI Overview란?** - Google과 다른 검색 엔진들이 AI를 사용해 검색 결과를 요약하는 방법
2. **지연 시간 문제** - 검색이 느린 이유와 더 빠르게 만드는 방법
3. **나만의 검색 엔진 구축** - 벡터 데이터베이스를 사용한 빠른 맞춤 검색 만들기
4. **AI와 커스텀 데이터 연결** - AI가 여러분의 콘텐츠를 기반으로 질문에 답하도록 만들기

## 사전 요구사항

- **Python 전문 지식 불필요!** 각 코드 블록을 단계별로 설명합니다
- **Upstage API 키**가 필요합니다 (발급 방법을 안내해 드립니다)
- 웹 검색에 대한 기본적인 이해 (Google 사용해 보셨죠? 그것으로 충분합니다!)

## 왜 중요한가요?

여러분이 기사, 문서, 또는 데이터 컬렉션을 가지고 있다고 상상해 보세요. AI가 다음을 할 수 있다면 얼마나 좋을까요:
- 콘텐츠를 즉시 검색 (1초 미만!)
- Google의 AI Overview처럼 지능적인 요약 제공
- 여러분의 특정 데이터를 기반으로 질문에 답변

오늘 바로 그것을 만들어 봅니다. 시작해볼까요!

---
# Part 1: AI Overview 이해하기
---

## Google AI Overview란?

Google에서 검색할 때, 가끔 링크 목록 대신 **상단에 멋진 요약**이 나타나는 것을 보신 적 있나요? 이것이 바로 **AI Overview** (이전에는 "Search Generative Experience" 또는 SGE로 불림)입니다.

### 작동 방식:
1. 질문을 입력합니다 (예: "달리기 잘하는 방법")
2. Google이 관련 웹페이지를 검색합니다
3. AI가 해당 페이지를 읽고 **요약**을 작성합니다
4. 여러 링크를 클릭하지 않고도 즉시 답변을 얻습니다!

### 예시: AI Overview 실제 작동

![Google AI Overview](./images/google_overview.png)

☝️ **위 이미지를 보세요**: "달리기 잘하는 방법"을 검색하면, Google의 AI가 올바른 자세, 훈련, 체력 관리에 대한 종합적인 답변을 제공합니다 - 모두 여러 출처에서 종합한 것입니다!


## 문제점: AI Overview가 항상 나타나지 않는다

여기에 함정이 있습니다 - **Google의 AI Overview는 모든 검색에서 작동하지 않습니다!**

### AI Overview가 나타나지 않는 경우는?
- **틈새 주제**: 전문적이거나 흔하지 않은 질문
- **특정 회사/제품 쿼리**: 소규모 비즈니스나 특정 제품에 대한 질문
- **지역 콘텐츠**: 주로 하나의 언어나 지역에 국한된 콘텐츠
- **신규 또는 새로운 주제**: 아직 널리 다루어지지 않은 것들

![No Google AI Overview](./images/no_google_overview.png)

☝️ **이 예시에서는**, 검색이 AI Overview를 트리거하지 않습니다 - 일반 검색 결과만 표시됩니다.

### 우리의 솔루션: 나만의 AI Overview 구축! 🛠️

다음과 같은 자체 AI Overview 시스템을 만들 수 있다면 어떨까요:
- **우리 자체 콘텐츠** (기사, 문서, 데이터베이스)와 함께 작동
- **데이터가 있는 모든 주제**에 대해 답변 제공
- Google을 기다리는 것보다 **더 빠름**

**바로 이것을 Upstage의 Solar-Pro2 모델을 사용하여 구축할 것입니다!**

---
# Part 2: 속도 문제
---

## 왜 검색이 느린가요? ⏱️

AI 기반 검색을 구축하면서 답답한 문제를 발견했습니다: **검색에 8초가 걸렸습니다!**

### 뒤에서 무슨 일이 일어나고 있나요?

AI에게 최신 정보가 필요한 질문을 할 때:

```
사용자 질문 → AI가 최신 데이터 필요 → 외부 검색 API 호출 → 대기... → 결과 수신 → 답변 생성
                                              ↑
                                    이 단계가 느립니다! (Tavily: ~1.5-2초)
```

![8 Sec for Search](./images/8_sec_for_search.png)

### 8초가 받아들일 수 없는 이유

- **사용자 경험**: 사람들은 2초 이내의 답변을 기대합니다
- **경쟁 열위**: Google의 AI Overview는 거의 즉시 나타납니다
- **다중 호출**: 여러 번 검색해야 하면 지연이 누적됩니다

### 병목 현상: 외부 검색 API

대부분의 AI 애플리케이션은 최신 웹 데이터를 얻기 위해 서드파티 검색 API(Tavily, Serper, Bing API 등)를 사용합니다. 이러한 API는 편리하지만 다음과 같은 이유로 **느립니다**:
1. Google 결과 스크래핑 (SERP 스크래핑)
2. 데이터 처리 및 포맷팅
3. 네트워크를 통해 결과 반환

**더 나은 솔루션이 필요합니다!**


## 검색 API 지연 시간: 현실 점검 📊

다양한 검색 API가 실제로 얼마나 빠른지(또는 느린지) 살펴봅시다. 이 데이터는 실제 측정값에서 가져온 것입니다:

![perplexity Search](./images/perplexity_search.avif)

### 숫자 이해하기

**p50, p75, p90, p95는 무엇을 의미하나요?**
- **p50 (중앙값)**: 요청의 50%가 이 시간보다 빠르게 완료됨
- **p75**: 요청의 75%가 이 시간보다 빠르게 완료됨
- **p90**: 요청의 90%가 이 시간보다 빠르게 완료됨
- **p95**: 요청의 95%가 이 시간보다 빠르게 완료됨 (5%만 더 느림)

### 검색 API 지연 시간 비교
*(AWS us-east-1에서 시작된 요청)*

| 제공업체 | p50 | p75 | p90 | p95 |
|----------|-----|-----|-----|-----|
| **Perplexity** | 358ms | 443ms | 604ms | 763ms |
| **Exa** | 1375ms | 1499ms | 1798ms | 2188ms |
| **Brave** | 513ms | 637ms | 728ms | 808ms |
| **SERP 기반*** | 1342ms | 1586ms | 1743ms | 1790ms |

\* 많은 검색 API 업체들이 결과의 기반으로 SERP 스크래핑을 사용합니다. 이러한 접근 방식의 성능을 측정하기 위해 Tavily가 제공하는 대표적인 Google SERP 기반 API를 사용합니다.

### 핵심 요점

1. **Perplexity가 가장 빠르지만** 그래도 검색당 ~350-750ms
2. **SERP 기반 API(Tavily 등)가 가장 느림** 1.3-1.8초
3. **이 중 어느 것도** LLM 처리와 결합했을 때 훌륭한 사용자 경험을 위해 **충분히 빠르지 않음**

**우리의 목표: 100ms 미만의 지연 시간을 가진 검색 엔진 구축!**

---
# Part 3: 구축해봅시다! AI Overview를 위한 Solar-Pro2 사용
---

## 간단한 웹 검색 RAG + Solar-Pro2

**RAG**는 **Retrieval-Augmented Generation(검색 증강 생성)**의 약자입니다. 간단한 개념을 위한 멋진 용어입니다:

```
RAG = 관련 정보 검색 + AI에게 전달 + 지능적인 답변 획득
```

### Solar-Pro2란?

**Solar-Pro2**는 다음을 수행할 수 있는 Upstage의 강력한 언어 모델입니다:
- 여러 언어(특히 한국어!)로 질문을 이해하고 답변
- 고품질 요약 및 설명 생성
- 웹 검색과 함께 작동하여 최신 답변 제공

### Solar 웹 검색 작동 방식

1. **질문하기** → "업스테이지가 1등일수 밖에 없는 이유"
2. **시스템이 웹 검색** → 관련 기사와 페이지 찾기
3. **Solar-Pro2가 결과 읽기** → 콘텐츠 이해
4. **AI Overview 반환** → 인용이 포함된 잘 정리된 답변!

> 💡 **더 알아보기**: 이 기능의 프롬프트 엔지니어링은 여기서 확인하세요:
> https://github.com/UpstageAI/solar-web-search/blob/main/prompts.py

### 직접 해봅시다! (아래 코드 설명)


### Step 1: 환경 설정

아래 코드는 다음을 수행합니다:

1. **라이브러리 가져오기** - 필요한 도구들 (`requests`로 API 호출 등)
2. **API 키 로드** - Upstage 서비스에 접근하기 위한 비밀 키 (`.env` 파일에 안전하게 저장됨)
3. **설정 구성** - 검색 엔진 위치와 사용할 컬렉션

> 🔑 **중요**: `UPSTAGE_API_KEY`가 포함된 `.env` 파일을 생성해야 합니다.
> 무료 API 키는 여기서 받으세요: https://console.upstage.ai/


In [7]:
import json
import requests
import time
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import concurrent.futures
from datetime import datetime, timedelta
import random
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv('.env', override=True)

# Configuration
SEARCH_ENGINE_BASE_URL = os.getenv('SEARCH_ENGINE_BASE_URL', 'https://search_api.toy.x.upstage.ai')
UPSTAGE_API_KEY = os.getenv('UPSTAGE_API_KEY')
COLLECTION_NAME = "sung_test"
BATCH_SIZE = 20
MAX_WORKERS = 1

if not UPSTAGE_API_KEY:
    raise ValueError("UPSTAGE_API_KEY not found in .env.local file")


print(f"Target: {SEARCH_ENGINE_BASE_URL}")
print(f"Collection: {COLLECTION_NAME}")

Target: https://search_api.toy.x.upstage.ai
Collection: sung_test


### Step 2: Solar-Pro2 검색 API 호출

이제 첫 번째 AI Overview 요청을 해봅시다! 코드가 하는 일:

**API 엔드포인트:**
```
https://solar-web-search.cosmic.upstage.ai/v1/chat/completions
```

**요청 구조:**
- `model`: "solar-pro2-search" - 웹 검색이 내장된 AI 모델
- `messages`: 질문 (채팅하듯이!)
- `stream`: False - 전체 응답을 기다림 (실시간 스트리밍은 True로 설정)

**수행 과정:**
1. Solar-Pro2에 질문을 보냅니다
2. Solar-Pro2가 웹에서 관련 정보를 검색합니다
3. [1], [2] 등의 인용이 포함된 종합적인 답변을 생성합니다
4. 깔끔하게 포맷된 AI Overview를 받습니다!


In [2]:
AI_OVERVIEW_URL = "https://solar-web-search.cosmic.upstage.ai/v1/chat/completions"

payload = {
  "model": "solar-pro2-search",
  "messages": [
    {
      "role": "user",
      "content": "업스테이지가 1등일수 밖에 없는 이유"
    }
  ],
  "stream": False,
}

headers = {
  'Authorization': f'Bearer {UPSTAGE_API_KEY}',
  'Content-Type': 'application/json'
}

response = requests.post(AI_OVERVIEW_URL, headers=headers, json=payload)

print(json.dumps(response.json(), indent=2))


{
  "id": "260c2baf-bf5d-4527-be3c-a76db55ea1e2",
  "object": "chat.completion",
  "created": 1764453087,
  "model": "solar-pro2-search",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "**\uc5c5\uc2a4\ud14c\uc774\uc9c0\ub294 \ud5c8\uae45\ud398\uc774\uc2a4 Open LLM \ub9ac\ub354\ubcf4\ub4dc\uc5d0\uc11c \uc138\uacc4 1\uc704\ub97c \ucc28\uc9c0\ud55c SOLAR \ubaa8\ub378\uacfc \uac15\ub825\ud55c B2B \uc194\ub8e8\uc158 \ud3ec\ud2b8\ud3f4\ub9ac\uc624\ub97c \ubc14\ud0d5\uc73c\ub85c \uae30\uc220\uc801 \uc6b0\uc704\uc640 \uc2e4\uc6a9\uc801\uc778 \uc801\uc6a9 \uac00\ub2a5\uc131\uc744 \uc785\uc99d\ud558\uba70 \uacbd\uc7c1\ub825\uc744 \uc720\uc9c0\ud558\uace0 \uc788\uc2b5\ub2c8\ub2e4 [1][2].**  \n\n### \uae30\uc220\uc801 \uc6b0\uc218\uc131  \n- **SOLAR \ubaa8\ub378\uc758 \uc131\uacfc**: \uc5c5\uc2a4\ud14c\uc774\uc9c0\uc758 \uc790\uccb4 \uac1c\ubc1c LLM\uc778 SOLAR\ub294 \ud5c8\uae45\ud398\uc774\uc2a4 Open LLM \ub9ac\ub354\ubcf4\ub4dc\uc5d0\uc

---
# Part 4: 현실로 만들기 - AskUp 크롬 확장 프로그램
---

## 코드에서 제품으로: AskUp AI Overview 🎉

이 기술을 누구나 사용할 수 있는 **Chrome 브라우저 확장 프로그램**으로 만들었습니다!

### 성능 결과

**첫 번째 검색: 2초 미만!**

![AskUp Overview](./images/askup_overview.png)

☝️ AI Overview가 Google 검색 결과에 바로 나타납니다 - 어떤 웹사이트도 방문할 필요 없습니다!

---

**캐싱 적용: 단 38밀리초! ⚡**

![AskUp Overview](./images/askup_overview_cache.png)

☝️ 이전에 본 질문이면, 캐시된 결과를 사용해 거의 즉시 응답할 수 있습니다.

### 캐싱 작동 방식

```
"AI란 무엇인가?" 첫 질문
  → 검색 + 생성 → ~1.5초

"AI란 무엇인가?" 두 번째 질문
  → 캐시된 답변 반환 → 38ms (40배 빠름!)
```

---

### 직접 사용해보세요! 🔗

**AskUp AI Overview 설치:**
https://chromewebstore.google.com/detail/askup-ai-overview/fdafojjfmhkfdhigkphhgkcahokajcgi 

![AskUp Overview QR](./images/askup_overview_qr.png)

QR 코드를 스캔하거나 링크를 클릭해서 Chrome 브라우저에 AI Overview를 추가하세요!


---
# Part 5: 1초 미만의 검색 엔진 구축
---

## 비밀: 벡터 데이터베이스 🗄️

**1초 미만의 검색**을 달성하려면 느린 외부 API에 의존하는 대신 자체 검색 엔진을 구축해야 합니다.

### 벡터 데이터베이스란?

전통적인 데이터베이스는 데이터를 행과 열로 저장합니다. **벡터 데이터베이스**는 텍스트의 *의미*를 포착하는 **수학적 표현** (벡터)으로 데이터를 저장합니다.

```
전통적 검색: "스마트폰 OS" → 정확한 단어 일치만
벡터 검색: "스마트폰 OS" → "모바일 운영체제", "안드로이드", "iOS"도 찾음
```

이것을 **시맨틱 검색**이라고 합니다 - 키워드가 아닌 의미로 검색!

### 적합한 벡터 데이터베이스 선택

여러 옵션을 평가했습니다. 분석 결과는 다음과 같습니다:

### 표 1: 벡터 데이터베이스 기능 비교 분석

| 데이터베이스 | 지연 시간 프로필 | 페이로드 처리 | 뉴스 적합성 | 양자화 기능 |
|----------|-----------------|------------------|----------------------|-----------------------|
| **Qdrant** ⭐ | 우수. Rust로 작성됨. 높은 동시 처리량. 벤치마크 결과 필터링 검색에 <25ms 지연. | 탁월. 벡터는 RAM에 유지하면서 디스크 페이로드(mmap) 지원. 페이로드 저장소가 벡터 인덱스와 분리됨. | 높음. 하이브리드 검색 및 다중 벡터(ColBERT) 기본 지원. 최신성을 위한 감쇠 함수 내장. | 고급. 재채점이 포함된 이진 양자화(BQ)로 최소한의 정확도 손실로 40배 속도 향상. |
| **Redis** | 가장 빠름(단순). 인메모리 K/V 스토어. 간단한 조회에 밀리초 미만. | 대용량 텍스트에 부적합. RAM에 큰 텍스트 블롭 저장은 비용이 높음. 지속성이 복잡하고 주요 설계 목표가 아님. | 중간. 빠르지만 전문 VDB에 비해 고급 필터링 유연성 부족. | 제한적. Qdrant에 비해 양자화 옵션이 덜 성숙함. |
| **Weaviate** | 양호. 강력한 하이브리드 검색 구현. 일부 시나리오에서 Qdrant보다 지연 시간이 높을 수 있음(320-550ms). | 양호. 객체 스토리지가 유연함. 스키마 적용이 더 엄격함. | 높음. 강력한 에코시스템과 하이브리드 지원, 다소 무거운 리소스 풋프린트. | 표준. 다양한 압축 방법 지원. |
| **Milvus** | 보통. 빠른 인덱싱, 소규모 고차원 설정에서 더 높은 지연/리소스 사용량. | 견고함. 대규모를 잘 처리하지만 운영 복잡성이 높음. | 중간. 하이브리드 검색 지원되지만 운영이 더 무거울 수 있음. | 표준. 다양한 압축 방법 지원. |

### 우리의 선택: Qdrant ⭐

**Qdrant**를 선택한 이유:
- **매우 빠름**: 필터링 검색에 <25ms
- **Rust 기반**: 높은 성능과 신뢰성
- **하이브리드 검색**: 최상의 결과를 위해 벡터 + 키워드 검색 결합
- **효율적인 스토리지**: 비용 부담 없이 대용량 텍스트 콘텐츠 처리 가능

## 1초 미만 지연 시간을 위한 엔지니어링 ⚡

목표가 **1초 미만**일 때, 모든 밀리초가 중요합니다! 시간 예산을 다음과 같이 배분합니다:

### 검색 파이프라인 (검색 시 수행되는 과정)

```
질문 → [인코딩] → [데이터베이스 검색] → [콘텐츠 가져오기] → [결과 순위 지정] → [응답 전송]
        ↓            ↓                    ↓                  ↓                   ↓
       20-50ms      30-60ms              10-20ms            20-300ms             10ms
```

### 표 3: 지연 시간 예산 분석

| 단계 | 작업 | 시간 예산 | 최적화 방법 |
|------|-----------|-------------|-----------------|
| 1️⃣ | **쿼리 인코딩** | 20-50ms | 빠른 임베딩 모델을 사용해 질문을 벡터로 변환 |
| 2️⃣ | **하이브리드 검색** | 30-60ms | Qdrant의 HNSW 인덱스로 유사 콘텐츠를 초고속 검색 |
| 3️⃣ | **페이로드 가져오기** | 10-20ms | SSD 메모리 매핑으로 데이터 복사 없이 콘텐츠 읽기 |
| 4️⃣ | **리랭킹** | 20-300ms | 관련성으로 결과 재정렬 (GPU: 20ms, CPU: 150-300ms) |
| 5️⃣ | **응답 조립** | 10ms | 결과를 JSON으로 패키징 |
| | **총계** | **~250-450ms** | **1000ms 제한보다 훨씬 낮음!** ✅ |

### 주요 최적화 기법 설명

- **HNSW 인덱스**: 모든 항목을 확인하지 않고 유사한 항목을 찾는 스마트 데이터 구조
- **이진 양자화**: 벡터를 압축하여 메모리 사용량을 줄이고 더 빠르게 검색 (40배 속도 향상!)
- **메모리 매핑 (mmap)**: RAM에 먼저 복사하지 않고 디스크에서 직접 데이터 읽기
- **ColBERT 리랭킹**: 컨텍스트를 이해하여 결과 순위를 더 잘 매기는 빠른 신경망

---
# Part 6: 실습 - 나만의 검색 엔진 구축
---

## Step 1: 문서 로딩 및 인덱싱 📚

이제 자체 검색 엔진을 구축해봅시다! 다음을 수행합니다:
1. 검색 인덱스에 **문서 추가**
2. 문서 **검색**
3. AI Overview를 위해 Solar-Pro2에 **연결**

### "인덱싱"이란?

도서관 카탈로그를 만드는 것과 같다고 생각하세요:
- **인덱스 없이**: 책을 찾으려면 모든 책을 일일이 봐야 함
- **인덱스 있으면**: 카탈로그에서 주제를 찾고 바로 맞는 서가로 이동

우리의 경우 인덱싱은 다음을 의미합니다:
1. 문서(기사, 뉴스 등) 가져오기
2. 검색 가능한 벡터로 변환
3. 빠른 검색을 위해 Qdrant에 저장

### 샘플 문서를 추가해봅시다!

### 인덱싱 코드 이해하기

아래 코드는 문서를 검색 엔진에 전송합니다. 분석해봅시다:

**API 엔드포인트:** `/index/{collection_name}`
- "컬렉션"은 문서의 폴더와 같습니다
- 다양한 목적으로 여러 컬렉션을 가질 수 있습니다

**문서 구조:**
```json
{
  "title": "기사 제목",           // 헤드라인
  "url": "https://...",          // 출처
  "content": "전체 텍스트...",    // 검색할 실제 콘텐츠
  "date": "2024-07-20"           // 발행일
}
```

**뒤에서 일어나는 일:**
1. API가 문서를 수신합니다
2. 각 문서가 벡터(임베딩)로 변환됩니다
3. 벡터 + 문서가 Qdrant에 저장됩니다
4. 초고속 검색 준비 완료! ⚡


In [3]:
import requests
import json

url = SEARCH_ENGINE_BASE_URL.rstrip('/') + "/index/" + COLLECTION_NAME
print(url)
headers = {
    "Authorization": f"Bearer {UPSTAGE_API_KEY}", # Using the API key from Colab secrets and stripping any extra whitespace/newlines
    "Content-Type": "application/json"
}

data = {
    "documents": [
      {
        "title": "Quantum Computing Breakthrough: New Algorithm Achieves Unprecedented Speed",
        "url": "https://www.techinnovationdaily.com/quantum-breakthrough-speed",
        "content": "Researchers at Quantum Labs have announced a groundbreaking new algorithm that significantly boosts the speed of quantum computations, potentially accelerating the development of new materials and drug discovery. The algorithm, detailed in 'Nature Physics', shows a 10x improvement over previous methods.",
        "date": "2024-07-20"
      },
      {
        "title": "AI Ethics Debate Intensifies as New Facial Recognition Software Raises Privacy Concerns",
        "url": "https://www.digitalfuturegazette.com/ai-ethics-privacy-concerns",
        "content": "A newly released facial recognition software by a prominent tech firm has ignited fresh debates around AI ethics and individual privacy. Critics argue the lack of regulation and potential for misuse poses significant societal risks, while proponents highlight its benefits for public safety.",
        "date": "2024-07-19"
      },
      {
        "title": "Sustainable Energy Sector Sees Record Investment in Q2 2024",
        "url": "https://www.greenpowertoday.com/sustainable-investment-q2",
        "content": "The second quarter of 2024 marked an all-time high for investments in sustainable energy projects globally. Venture capital and private equity firms poured billions into solar, wind, and geothermal technologies, signaling a strong market shift towards renewable resources.",
        "date": "2024-07-18"
      },
      {
        "title": "Major Tech Company Announces Layoffs Amid Restructuring Efforts",
        "url": "https://www.globaltechchronicle.com/company-layoffs-restructuring",
        "content": "One of the world's largest technology companies has announced a significant round of layoffs as part of a broader restructuring initiative. The move is aimed at streamlining operations and refocusing on core product lines, affecting thousands of employees across various departments.",
        "date": "2024-07-17"
      },
      {
        "title": "New Smartphone OS Unveiled: Focus on AI Integration and User Privacy",
        "url": "https://www.mobileworldweekly.com/new-os-ai-privacy",
        "content": "A highly anticipated new smartphone operating system was unveiled today, promising deep AI integration for personalized user experiences alongside enhanced privacy features. Developers are eager to begin building applications for the new platform, which aims to redefine mobile computing.",
        "date": "2024-07-21"
      }
    ]
}

# Adding verify=False due to previous SSL certificate issues
response = requests.post(url, headers=headers, data=json.dumps(data), verify=False)

print(f"Status Code: {response.status_code}")
print(f"Response Body:\n{json.dumps(response.json(), indent=2)}")

https://search_api.toy.x.upstage.ai/index/themilk
Status Code: 200
Response Body:
{
  "status": "success",
  "count": 5
}


## Step 2: 검색 엔진 테스트 🔍

이제 문서를 인덱싱했으니 검색해봅시다!

### 검색 작동 방식

```
질문: "새로운 스마트폰 OS는?"
       ↓
[벡터로 변환] → [Qdrant에서 유사 벡터 찾기] → [일치하는 문서 반환]
       ↓
결과: 스마트폰 운영체제, AI 통합, 프라이버시 기능에 관한 기사들...
```

### 검색 응답 이해하기

검색은 **결과** 목록을 반환하며, 각각 다음을 포함합니다:
- `title`: 기사 헤드라인
- `snippet`: 콘텐츠의 짧은 미리보기
- `content`: 전체 텍스트
- `score`: 관련성 (1.0 = 완벽한 일치)
- `url`: 출처 링크
- `date`: 발행일

**"What's new smartphone OS?"로 검색해봅시다:**

In [6]:
import requests
import json

search_endpoint = SEARCH_ENGINE_BASE_URL.rstrip('/') + "/search/" + COLLECTION_NAME
print(url)
headers = {
    "Authorization": f"Bearer {UPSTAGE_API_KEY}", # Using the api_key obtained previously
    "Content-Type": "application/json"
}

data = {
    "query": "What's new smart phone OS?",
    "limit": 5
}

# Adding verify=False due to previous SSL certificate issues
response = requests.post(search_endpoint, headers=headers, data=json.dumps(data), verify=False)

print(f"Status Code: {response.status_code}")
print(f"Response Body: {response.json()}")
# --- END MODIFIED CODE ---

https://search_api.toy.x.upstage.ai/search/themilk
Status Code: 200
Response Body: {'results': [{'title': 'New Smartphone OS Unveiled: Focus on AI Integration and User Privacy', 'url': 'https://www.mobileworldweekly.com/new-os-ai-privacy', 'snippet': 'A highly anticipated new smartphone operating system was unveiled today, promising deep AI integration for personalized user experiences alongside enhanced privacy features. Developers are eager to begin building applications for the new platform, which aims to redefine mobile computing.', 'content': 'A highly anticipated new smartphone operating system was unveiled today, promising deep AI integration for personalized user experiences alongside enhanced privacy features. Developers are eager to begin building applications for the new platform, which aims to redefine mobile computing.', 'date': '2024-07-21', 'last_updated': None, 'full_content': 'A highly anticipated new smartphone operating system was unveiled today, promising deep AI in

---
# Part 7: 마법 - 여러분의 데이터로 AI Overview
---

## Step 3: Solar-Pro2를 우리 검색 엔진에 연결 🪄

모든 것이 합쳐지는 부분입니다! Solar-Pro2가 공개 웹 대신 **우리 자체 검색 엔진**을 사용하도록 연결합니다.

### 핵심 설정

```python
"web_search_options": {
    "external_search_engine_only": True,  # 우리 검색 엔진만 사용
    "servers": [{
        "url": "our_search_endpoint",     # Qdrant 기반 검색을 가리킴
        "key": "our_api_key"
    }]
}
```

### 이것이 가능하게 하는 것

1. **프라이버시**: 데이터가 여러분의 통제를 벗어나지 않음
2. **맞춤화**: 여러분의 콘텐츠를 기반으로 AI가 답변
3. **속도**: 1초 미만 응답 (느린 외부 API 없음!)
4. **정확성**: 여러분의 특정 도메인 지식에 기반한 답변

### 직접 해봅시다!

인덱싱된 콘텐츠를 사용해 "윤송이 박사님 강의"에 대해 물어봅시다:

In [10]:
AI_OVERVIEW_URL = "https://solar-web-search.cosmic.upstage.ai/v1/chat/completions"

payload = {
  "model": "solar-pro2-search",
  "messages": [
    {
      "role": "user",
      "content": "윤송이 박사님 강의"
    }
  ],
  "stream": False,
  "web_search_options": {
    "search_context_size": "medium",
    "external_search_engine_only": True,
    "servers": [
      {
        "url": search_endpoint,
        "key": UPSTAGE_API_KEY
      }
    ]
  }
}

headers = {
  'Authorization': f'Bearer {UPSTAGE_API_KEY}',
  'Content-Type': 'application/json'
}

print(f"Querying Solar API with external search: {search_endpoint}...")
response = requests.post(AI_OVERVIEW_URL, headers=headers, json=payload)

print(json.dumps(response.json(), indent=2))


Querying Solar API with external search: https://search_api.toy.x.upstage.ai/search/themilk...
{
  "id": "6d3552e5-536c-4dcf-8902-62e1b2c7ddf3",
  "object": "chat.completion",
  "created": 1764422960,
  "model": "solar-pro2-search",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "**\uc724\uc1a1\uc774 \ubc15\uc0ac\ub294 AI\ub97c \uc778\ub958 \uc9c4\ud654\uc758 \ubcc0\uace1\uc810\uc73c\ub85c \ubcf4\uba70, \ubd88\uacfc \uc5b8\uc5b4\uc5d0 \uc774\uc740 \uc138 \ubc88\uc9f8 \uadfc\uc6d0\uc801 \ubcc0\ud654 \ub3c4\uad6c\ub85c \uac15\uc870\ud588\uc2b5\ub2c8\ub2e4 [1][4][8].**  \n\n### \uc0c1\uc138 \uc124\uba85  \n1. **AI\ub97c \uc778\ub958 \ubb38\uba85\uc758 \uc138 \ubc88\uc9f8 \ubd88\uae38\ub85c \uaddc\uc815**  \n   - \uc724\uc1a1\uc774 PVP \ub300\ud45c\ub294 2026 \uae30\uc870\uc5f0\uc124\uc5d0\uc11c \"AI\ub294 \ub300\ud55c\ubbfc\uad6d\uc758 \uc138 \ubc88\uc9f8 \ubd88\"\uc774\ub77c\uace0 \uc8fc\uc7a5\ud558\uba70, \ubd88\uacfc \uc5b8\uc5

---
# Part 8: 실제 적용 사례 - 더밀크 🥛
---

## 사례 연구: 더밀크를 위한 AI 기반 뉴스 검색

**더밀크**는 한국의 비즈니스/테크 뉴스 플랫폼입니다. 우리의 검색 엔진을 통합하여 AI 기능을 구동합니다!

### 검색 경험

![Themilk search](./images/themilk_search.png)

☝️ 사용자들이 비즈니스 뉴스에 대해 질문하고 더밀크의 기사 데이터베이스를 기반으로 AI가 생성한 답변을 받을 수 있습니다 - Google의 AI Overview와 똑같지만, 그들의 특정 콘텐츠를 위한 것입니다!

---

### 비즈니스 임팩트

![Themilk sales](./images/themilk_sales.png)

이 기술이 가능하게 하는 것:
- **더 나은 사용자 참여**: 사용자가 더 빠르게 답변을 찾음
- **콘텐츠 수익화**: AI 답변이 구독을 유도
- **경쟁 우위**: 주요 플랫폼이 틈새 콘텐츠에 제공하지 않는 기능

---

## 🎉 축하합니다!

다음을 배웠습니다:
1. ✅ AI Overview가 무엇이고 왜 중요한지 이해
2. ✅ 외부 검색 API의 지연 시간 문제 파악
3. ✅ 적합한 벡터 데이터베이스 선택 (Qdrant!)
4. ✅ 문서 컬렉션 구축 및 인덱싱
5. ✅ 시맨틱 검색으로 문서 검색
6. ✅ Solar-Pro2를 자체 검색 엔진에 연결
7. ✅ 나만의 AI Overview 경험 만들기!

### 다음 단계

- 🔗 API 키 받기: https://console.upstage.ai/
- 📚 Solar-Pro2 살펴보기: https://developers.upstage.ai/
- 🧩 AskUp 확장 프로그램 사용해보기: Chrome 웹 스토어에서 설치하세요!

**즐거운 개발 되세요! 🚀**
